# Lib import

In [ ]:
import sys
import json
import pandas as pd
from datasets import load_metric

# Load dataset

In [ ]:
def load_data(dts):
    path = f'MuP_dataset/{dts}_complete.jsonl'
    try:
        with open(path, 'r') as json_file:
            json_list = list(json_file)
        col_name = ["paper_id","summary"]
    except:
        print(f"Warning: Did not load dataset from {path}")
        return
    summary_df = pd.DataFrame(columns=col_name)
    for json_str in json_list[:]:
        result = json.loads(json_str)
        df = pd.DataFrame([[result["paper_id"], result["summary"]]], columns=col_name)
        summary_df = pd.concat([summary_df,df])
    return summary_df

def split_sum_num(df):
    num_paper = df.groupby(['paper_id']).count()
    num_paper['num_paper'] = 1
    num_paper.groupby(['summary']).count()
    num_paper = df.groupby(['paper_id']).count()
    num_paper['num_paper'] = 1
    num_paper.groupby(['summary']).count()
    num_paper.drop('num_paper', inplace=True,axis=1)
    num_paper.sort_values(["summary"])

    df_list = []
    for i in range(0, max(num_paper['summary'])):
        paper_id = (list((num_paper[num_paper["summary"]==(i+1)]).index))
        df_i = df[df.paper_id.isin(paper_id)].sort_values("paper_id")
        df_list.append(df_i.groupby('paper_id').apply(lambda df_: df_[['summary']].values.flatten()).apply(pd.Series).reset_index())
        
    return df_list

In [ ]:
dts = "training"
summary_df = load_data(dts)

In [ ]:
df_list = split_sum_num(summary_df)

In [ ]:
df_list[3]

# Score calculation

In [ ]:
from evaluate import load

In [ ]:
def n_scores(df, subscore_col):
    n = len(df.columns)-1
    pairs = [f'{i}-{j}' for i in range(n) for j in range(i+1, n)]
    col = pd.MultiIndex.from_product([subscore_col, pairs])
    scores = pd.DataFrame(columns=col)
    scores.insert(0, "paper_id", df["paper_id"])
    # scores.set_index("paper_id", inplace=True)
    return scores

## ROUGE Score

In [ ]:
rougescore = load("rouge")
rougescore = load_metric("rouge")

def rouge_cal(df):
    n = len(df.columns)-1
    print(f"Calculating ROUGE on {n} summaries")

    rouge_list = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
    score_list = ['precision', 'recall', 'fmeasure']

    df_score_dict = {}
    mux = pd.MultiIndex.from_product([["summary"],(list(df.columns))[1:]])
    df_score = pd.DataFrame(columns=mux)
    df_score.insert(0, "paper_id", df["paper_id"])
    for col in df:
        if col != 'paper_id':
            df_score[('summary', col)] = df[col]
    df_score = df_score.merge(n_scores(df, score_list), left_on='paper_id', right_on='paper_id')
    for r in rouge_list:
        df_score_dict[r] = df_score

    df_len = len(df)
    for idx, row in df.iterrows():
        sys.stdout.write(f"\r{idx+1}/{df_len}")
        sys.stdout.flush()
        for i in range(n):
            for j in range(i+1,n):
                pair = f'{i}-{j}'
                score = rougescore.compute(predictions=[row[i]], references=[row[j]], use_stemmer=False)
                for r in rouge_list:
                    df_score_dict[r].loc[idx, ('precision', pair)] = ((score[r]).low).precision
                    df_score_dict[r].loc[idx, ('recall', pair)] = ((score[r]).low).recall
                    df_score_dict[r].loc[idx, ('fmeasure', pair)] = ((score[r]).low).fmeasure

    return df_score_dict

In [ ]:
for n, df in enumerate(df_list):
    if n+1 > 1:
        dict_result = rouge_cal(df)
        for key, val in dict_result.items():
            val.to_csv(f"visualization_data/rouge-between-sum/{key}/{dts}_{key}_{n+1}sum.csv")
dict_result['rouge1']

## BERT Score

In [228]:
bertscore = load("bertscore")
def bertscore_cal(df):
    n = len(df.columns)-1
    print(f"Calculating BERTScore on {n} summaries")
    
    score_list = ['precision', 'recall', 'f1']

    mux = pd.MultiIndex.from_product([["summary"],(list(df.columns))[1:]])
    df_score = pd.DataFrame(columns=mux)
    df_score.insert(0, "paper_id", df["paper_id"])
    for col in df:
        if col != 'paper_id':
            df_score[('summary', col)] = df[col]
    df_score = df_score.merge(n_scores(df, score_list), left_on='paper_id', right_on='paper_id')

    for i in range(n):
        for j in range(i+1,n-1):
            pair = f'{i}-{j}'
            summary1 = list(df_score.loc[:, ('summary', i)])
            summary2 = list(df_score.loc[:, ('summary', j)])
            print(pair)
            result = bertscore.compute(predictions=summary1, references=summary2, lang="en", rescale_with_baseline=True)
            for score in score_list:
                df_score.loc[:, (score, pair)] = result[score]

    return df_score
    
bertscore_cal(df_list[2][:3])

Calculating BERTScore on 3 summaries
0-1


/var/folders/zw/q5kx3f693rgfk0h2y_jgmhnm0000gn/T/ipykernel_26093/3945739358.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_score = df_score.merge(n_scores(df, score_list), left_on='paper_id', right_on='paper_id')
/var/folders/zw/q5kx3f693rgfk0h2y_jgmhnm0000gn/T/ipykernel_26093/3945739358.py:25: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_score.loc[:, (score, pair)] = result[score]


paper_id  \
                                                 
0  SP:001ae7379191bb40fc356a37deb2f0ffc4426e52   
1  SP:0045d9a733e9d3db6e42c4fcadaa7dc57f7b5004   
2  SP:0055dca69c153ed21b420741c479a2ef00be2ef6   

                                             summary  \
                                                   0   
0  This paper proposes EgoTR for cross-view geo-l...   
1  This paper considers the problem of generaliza...   
2  This paper compares the so-called "State Evolu...   

                                                      \
                                                   1   
0  The paper tackles the cross-view geo-localizat...   
1  This paper aims to design a counterfactual rei...   
2  This paper considers the state evolution equat...   

                                                     precision            \
                                                   2       0-1  0-2  1-2   
0  This paper proposes to use transformer for cro...  0.224304  NaN  NaN   
1  The paper presents a method to improve the gen... -0.104706  NaN  NaN   
2  This paper provides derivations of equivalence...  0.054633  NaN  NaN   

     recall                  f1            
        0-1  0-2  1-2       0-1  0-2  1-2  
0  0.056118  NaN  NaN  0.140213  NaN  NaN  
1  0.213069  NaN  NaN  0.050674  NaN  NaN  
2  0.191037  NaN  NaN  0.123345  NaN  NaN

In [ ]:
for n, df in enumerate(df_list):
    if n+1 > 1:
        result = bertscore_cal(df)
        val.to_csv(f"visualization_data/bertscore-between-sum/{dts}_bertscore_{n+1}sum.csv")